In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

In [ ]:
# Ekert91-Attacker.py

import math
import random
from qiskit import QuantumCircuit, Aer, execute

def create_singlet():
    """
    Create a 2-qubit circuit that prepares the singlet state:
    |ψ⁻⟩ = (|01⟩ - |10⟩) / √2.
    """
    qc = QuantumCircuit(2)
    qc.x(1)          # Prepare |0>|1>
    qc.h(0)
    qc.cx(0, 1)
    qc.z(1)          # Introduce the minus sign
    return qc

def measure_in_basis(qc, qubit, basis):
    """
    Rotate the state on 'qubit' so that a subsequent Z measurement 
    is equivalent to measuring in the specified basis.
    
    Allowed bases:
      - 'Z': no rotation,
      - 'X': apply a Hadamard,
      - 'V': approximated here via an ry rotation.
    """
    if basis == 'Z':
        pass
    elif basis == 'X':
        qc.h(qubit)
    elif basis == 'V':
        qc.ry(-math.pi/2, qubit)  # Approximation for V basis.
    else:
        raise ValueError("Unknown basis provided.")

def basis_selector():
    """
    Use a quantum coin toss to select a measurement basis with bias:
         (1/√3)|0⟩ + (√2/√3)|1⟩.
    Outcome 0 occurs with probability 1/3; outcome 1 with probability 2/3.
    """
    theta = 2 * math.acos(1/math.sqrt(3))
    qc = QuantumCircuit(1, 1)
    qc.ry(theta, 0)
    qc.measure(0, 0)
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1)
    result = job.result()
    counts = result.get_counts(qc)
    outcome = list(counts.keys())[0]
    return int(outcome)

def simulate_round():
    """
    Simulate one round of the Ekert91 protocol with an attacker intercepting Bob's qubit.
    
    Steps:
      1. Prepare the entangled singlet state.
      2. The attacker intercepts Bob's qubit:
         - The attacker measures Bob’s qubit in a random basis (simulated here by simply 
           disturbing the state) and then resets it.
         - For simplicity, after the reset Bob's qubit is prepared in the |0⟩ state.
      3. Alice and Bob choose their measurement bases using the biased coin toss.
      4. Apply the corresponding rotations and measure.
    """
    qc = create_singlet()
    
    # --- Attacker intervention on Bob's qubit ---
    # The attacker measures Bob's qubit in a random basis and resets it.
    attacker_basis = random.choice(['Z', 'X'])
    measure_in_basis(qc, 1, attacker_basis)
    # Here, we measure Bob's qubit to simulate the attacker's action.
    qc.measure(1, 0)  # Note: using an extra classical bit (this measurement is just for simulation).
    qc.reset(1)       # Reset Bob's qubit to |0⟩.
    # (A more detailed simulation could conditionally prepare Bob's qubit based on the measurement outcome.)
    
    # --- End attacker intervention ---
    
    # Basis selection for Alice and Bob after the attack.
    alice_coin = basis_selector()  # 0 or 1
    bob_coin   = basis_selector()    # 0 or 1
    
    alice_basis = 'Z' if alice_coin == 0 else 'X'
    bob_basis   = 'Z' if bob_coin == 0 else 'V'
    
    # Apply the rotations so that Z measurement gives the desired observable.
    measure_in_basis(qc, 0, alice_basis)
    measure_in_basis(qc, 1, bob_basis)
    
    qc.measure_all()
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=1)
    result = job.result()
    counts = result.get_counts(qc)
    outcome = list(counts.keys())[0]
    
    # Extract results (note the bit ordering: rightmost is qubit 0, next is qubit 1)
    alice_result = int(outcome[-1])
    bob_result   = int(outcome[-2])
    
    return alice_basis, bob_basis, alice_result, bob_result

def simulate_protocol(num_rounds=100):
    """
    Run the Ekert91 protocol with an attacker for a given number of rounds.
    Returns a list of tuples: (alice_basis, bob_basis, alice_result, bob_result)
    """
    results = []
    for _ in range(num_rounds):
        results.append(simulate_round())
    return results

if __name__ == "__main__":
    rounds = simulate_protocol(num_rounds=100)
    print("Example round (with attacker):", rounds[0])
